In [1]:
import json
import bz2

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import sklearn

In [3]:
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

In [5]:
train = pd.read_csv('Downloads/features.csv')

In [6]:
test = pd.read_csv('Downloads/features_test.csv')

In [7]:
train_results = train[['duration',
       'radiant_win', 'tower_status_radiant', 'tower_status_dire',
       'barracks_status_radiant', 'barracks_status_dire']]

In [78]:
test.columns

Index(['r1_hero', 'r1_level', 'r1_xp', 'r1_gold', 'r1_lh', 'r1_kills',
       'r1_deaths', 'r1_items', 'r2_hero', 'r2_level', 'r2_xp', 'r2_gold',
       'r2_lh', 'r2_kills', 'r2_deaths', 'r2_items', 'r3_hero', 'r3_level',
       'r3_xp', 'r3_gold', 'r3_lh', 'r3_kills', 'r3_deaths', 'r3_items',
       'r4_hero', 'r4_level', 'r4_xp', 'r4_gold', 'r4_lh', 'r4_kills',
       'r4_deaths', 'r4_items', 'r5_hero', 'r5_level', 'r5_xp', 'r5_gold',
       'r5_lh', 'r5_kills', 'r5_deaths', 'r5_items', 'd1_hero', 'd1_level',
       'd1_xp', 'd1_gold', 'd1_lh', 'd1_kills', 'd1_deaths', 'd1_items',
       'd2_hero', 'd2_level', 'd2_xp', 'd2_gold', 'd2_lh', 'd2_kills',
       'd2_deaths', 'd2_items', 'd3_hero', 'd3_level', 'd3_xp', 'd3_gold',
       'd3_lh', 'd3_kills', 'd3_deaths', 'd3_items', 'd4_hero', 'd4_level',
       'd4_xp', 'd4_gold', 'd4_lh', 'd4_kills', 'd4_deaths', 'd4_items',
       'd5_hero', 'd5_level', 'd5_xp', 'd5_gold', 'd5_lh', 'd5_kills',
       'd5_deaths', 'd5_items', 'first_blood

In [8]:
df = train.drop(['duration',
       'radiant_win', 'tower_status_radiant', 'tower_status_dire',
       'barracks_status_radiant', 'barracks_status_dire'],axis = 1)

In [9]:
df = df.drop(['dire_bottle_time','radiant_bottle_time'],axis = 1)

In [10]:
df = df.drop(['start_time',],axis = 1)

In [11]:
types = [28550, 12718, 55962]

In [12]:
test = test.drop(['start_time','dire_bottle_time','radiant_bottle_time'],axis = 1)

In [13]:
df= df.drop('match_id',axis = 1)
test = test.drop('match_id',axis = 1)

In [83]:
y = train_results['radiant_win']

In [15]:
df['dire_first_ward_time'].fillna(1000,inplace=True)

In [16]:
test['dire_first_ward_time'].fillna(1000,inplace=True)
test['radiant_first_ward_time'].fillna(1000,inplace=True)
test['first_blood_time'].fillna(1000,inplace=True)
test['first_blood_team'].fillna(2,inplace=True)
test['first_blood_player1'].fillna(0,inplace=True)
test['radiant_courier_time'].fillna(1000,inplace=True)
test['dire_courier_time'].fillna(1000,inplace=True)
test['radiant_flying_courier_time'].fillna(1000,inplace=True)
test['dire_flying_courier_time'].fillna(1000,inplace=True)
test['first_blood_player2'].fillna(0,inplace=True)

In [17]:
test = test.drop('lobby_type',axis=1)

In [18]:
df['radiant_first_ward_time'].fillna(1000,inplace=True)

In [104]:
list_of_nans = []
for i in df.count().values:
    if i!= 97230:
        for b in df.count().index:
            if df.count()[b] == i:
                list_of_nans.append(b)

In [79]:
test

,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,...,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,93,4,1103,1089,8,0,1,9,102,3,...,3,0,12.0,-86.0,272.0,3,4,2,0,118.0
1,20,2,556,570,1,0,0,9,6,4,...,2,2,-29.0,-54.0,1000.0,3,2,2,1,16.0
2,112,2,751,808,1,0,0,13,26,2,...,3,1,-22.0,-87.0,186.0,1,3,3,0,-34.0
3,27,3,708,903,1,1,1,11,91,2,...,2,2,-49.0,-89.0,210.0,3,4,2,1,-26.0
4,39,4,1259,661,4,0,0,9,93,5,...,2,0,36.0,-86.0,180.0,1,3,2,1,-33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17172,11,5,2054,1941,27,0,1,8,28,4,...,4,1,8.0,-87.0,1000.0,4,3,2,1,-33.0
17173,3,3,748,605,1,0,0,12,22,3,...,2,1,-1.0,-85.0,184.0,2,3,4,1,-18.0
17174,85,2,575,499,0,0,0,8,102,3,...,2,0,20.0,-88.0,239.0,4,4,4,0,-36.0
17175,7,4,1844,1176,8,1,2,8,32,3,...,2,0,-28.0,-83.0,1000.0,1,4,1,0,1000.0


In [19]:
df['first_blood_time'].fillna(1000,inplace=True)
df['first_blood_team'].fillna(2,inplace=True)
df['first_blood_player1'].fillna(0,inplace=True)

In [20]:
df['radiant_courier_time'].fillna(1000,inplace=True)
df['dire_courier_time'].fillna(1000,inplace=True)

In [21]:
df['radiant_flying_courier_time'].fillna(1000,inplace=True)
df['dire_flying_courier_time'].fillna(1000,inplace=True)

In [22]:
df['first_blood_player2'].fillna(0,inplace=True)

In [23]:
df = df.drop('lobby_type',axis = 1)

In [84]:
X_train, X_test, y_train, y_test = train_test_split(df, y,)

In [85]:
X_train

,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,...,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
49751,21,4,1759,1196,13,0,0,7,30,3,...,1,2,19.0,-87.0,206.0,4,5,3,1,-70.0
67267,73,5,2048,2406,21,0,0,4,68,2,...,2,1,0.0,-87.0,1000.0,4,4,2,0,17.0
87616,60,4,1563,1153,16,0,0,10,31,2,...,3,1,-33.0,-78.0,1000.0,1,4,2,0,20.0
35187,62,4,1708,820,5,0,0,6,78,5,...,4,1,-15.0,-79.0,1000.0,2,3,2,1,-6.0
8574,38,4,1754,1346,14,1,2,7,3,3,...,2,1,-47.0,-88.0,266.0,3,2,2,1,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89174,73,5,1933,2157,15,0,0,5,62,3,...,2,1,-34.0,-87.0,1000.0,5,4,4,3,21.0
10774,79,3,794,628,0,0,0,9,39,5,...,3,1,1.0,-84.0,290.0,2,3,2,1,-18.0
12493,11,4,1842,1161,16,0,1,6,7,2,...,3,0,2.0,-83.0,182.0,4,4,2,1,-35.0
46467,1,3,1117,1222,15,0,0,3,11,5,...,0,0,1000.0,-85.0,193.0,3,5,2,0,-11.0


In [136]:
from sklearn.model_selection import cross_val_score

In [25]:
clf = GradientBoostingClassifier().fit(X_train,y_train)

In [26]:
np.argsort(clf.feature_importances_)

array([48, 64, 62, 57, 23, 24, 25, 40, 56, 54, 29, 53, 96, 47, 41, 55, 17,
       65, 15,  1, 88, 83,  6, 69, 80,  7, 73, 72, 71, 13, 14,  9, 37, 45,
       22, 32, 89, 95,  0, 84,  8, 49, 18, 46, 63,  5, 61, 31, 33, 21, 78,
       79, 16, 91, 38, 70, 10, 42, 77, 39, 86, 52, 30, 26,  2, 66, 90, 50,
       97, 76, 34, 93, 74, 60, 28, 44,  4, 92, 20, 12, 58, 36, 68, 85, 94,
       81, 87, 82, 59, 67, 19, 35, 75, 27, 43, 51,  3, 11])

In [27]:
X_train.columns[ [12, 58, 36, 68, 85, 94,81, 87, 82, 59, 67, 19, 35, 75, 27, 43, 51,  3, 11]]

Index(['r2_lh', 'd3_xp', 'r5_lh', 'd4_lh', 'radiant_flying_courier_time',
       'dire_boots_count', 'first_blood_team', 'radiant_boots_count',
       'first_blood_player1', 'd3_gold', 'd4_gold', 'r3_gold', 'r5_gold',
       'd5_gold', 'r4_gold', 'd1_gold', 'd2_gold', 'r1_gold', 'r2_gold'],
      dtype='object')

In [146]:
clf.feature_importances_

array([0.        , 0.        , 0.0007586 , 0.08216374, 0.00452853,
       0.00023534, 0.00037687, 0.        , 0.00019712, 0.00024948,
       0.00109611, 0.07708219, 0.00921716, 0.        , 0.        ,
       0.        , 0.00044767, 0.        , 0.00052361, 0.06934463,
       0.0019335 , 0.        , 0.00102583, 0.00156093, 0.00021383,
       0.        , 0.00350837, 0.07209579, 0.00763911, 0.0003212 ,
       0.00334985, 0.00029003, 0.        , 0.        , 0.00313403,
       0.07496625, 0.01095611, 0.00060288, 0.00114023, 0.00048525,
       0.        , 0.00024555, 0.00337821, 0.07579033, 0.00550602,
       0.        , 0.00140011, 0.        , 0.00020751, 0.        ,
       0.0019026 , 0.08440118, 0.00593092, 0.00028414, 0.        ,
       0.        , 0.        , 0.        , 0.00412093, 0.06307609,
       0.00704997, 0.00027533, 0.00071678, 0.00049131, 0.        ,
       0.        , 0.00275624, 0.07141367, 0.01133617, 0.        ,
       0.        , 0.00035945, 0.00022365, 0.        , 0.00442

In [28]:
clf.score(X_test,y_test)

0.6499917722560474

In [142]:
cv_scores

array([0.68445404, 0.65268353, 0.66317088, 0.68260333, 0.65946946,
       0.64281308, 0.63602714, 0.6668723 , 0.65484269, 0.6369525 ,
       0.60919186, 0.66132017, 0.64096237, 0.63972856, 0.63745757,
       0.63190373, 0.62037037, 0.61512346, 0.62407407, 0.64537037,
       0.6367284 , 0.65555556, 0.65      , 0.63919753, 0.63024691,
       0.64475309, 0.63364198, 0.65462963, 0.65617284, 0.67932099])

In [148]:
np.mean(cv_scores)

0.6461879470664056

In [29]:
X_train.shape

(72922, 98)

In [35]:
X_train.iloc[0].values.shape

(98,)

In [46]:
chunks_of_df = np.array_split(df, 3241)

In [86]:
chunks_of_df = np.array_split(X_train, 3640)

In [47]:
chunks_of_df[0].shape

(30, 98)

In [48]:
chunks_of_y = np.array_split(y, 3241)

In [87]:
chunks_of_y = np.array_split(y_train, 3640)

In [89]:
chunks_of_test_x = np.array_split(X_test, 1215)

In [90]:
chunks_of_test_y = np.array_split(y_test, 1215)

In [101]:
X = chunks_of_df[0].values
X =torch.from_numpy(X).float()
output = net(X)
output

tensor([[-0.4753, -0.9721],
        [-0.9296, -0.5021],
        [-1.1393, -0.3857],
        [-1.1428, -0.3841],
        [-1.0186, -0.4480],
        [-0.3454, -1.2308],
        [-0.7164, -0.6705],
        [-0.7257, -0.6616],
        [-0.0631, -2.7939],
        [-0.7291, -0.6585],
        [-0.8986, -0.5228],
        [-0.4397, -1.0335],
        [-0.7164, -0.6705],
        [-0.6872, -0.6991],
        [-0.7601, -0.6304],
        [-0.7777, -0.6152],
        [-0.6784, -0.7081],
        [-0.8565, -0.5528],
        [-1.0383, -0.4370],
        [-0.6980, -0.6884],
        [-0.7546, -0.6353]], grad_fn=<LogSoftmaxBackward>)

In [102]:
y = chunks_of_y[index_of_chunk].values
y = torch.from_numpy(y).float()
y.long()


tensor([1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1])

In [115]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(98,128)
        self.fc2 = nn.Linear(128,128)
        self.fc3 = nn.Linear(128,64)
        self.fc4 = nn.Linear(64,2)
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        '''h_relu = self.fc1(x).clamp(min=0)
        y_pred = self.fc2(h_relu)
        y_pred = self.fc3(y_pred)
        y_pred = self.fc4(y_pred)'''
        #return y_pred
        #return x
        return F.log_softmax(x,dim = 1)
net = Net()

In [116]:
optimiser = torch.optim.Adam(net.parameters(), lr=0.001)

EPOCHS = 200
#loss_fn = torch.nn.MSELoss(reduction='sum')
for epoch in range(EPOCHS):
    for index_of_chunk in range(1216):
        X = chunks_of_df[index_of_chunk].values
        y = chunks_of_y[index_of_chunk].values
        X =torch.from_numpy(X).float()
        y = torch.from_numpy(y).float()
        net.zero_grad()
        output = net(X)
        loss = F.nll_loss(output, y.long())
        #loss = torch.nn.MSELoss(output, y )
        #loss = criterion(output,y)
        #loss = loss_fn(output,y)
        loss.backward()
        optimiser.step()
    print(loss)
        

tensor(0.6583, grad_fn=<NllLossBackward>)
tensor(0.6652, grad_fn=<NllLossBackward>)
tensor(0.7302, grad_fn=<NllLossBackward>)
tensor(0.7062, grad_fn=<NllLossBackward>)
tensor(0.6947, grad_fn=<NllLossBackward>)
tensor(0.7121, grad_fn=<NllLossBackward>)
tensor(0.6865, grad_fn=<NllLossBackward>)
tensor(0.6913, grad_fn=<NllLossBackward>)
tensor(0.6875, grad_fn=<NllLossBackward>)
tensor(0.7076, grad_fn=<NllLossBackward>)
tensor(0.6796, grad_fn=<NllLossBackward>)
tensor(0.6749, grad_fn=<NllLossBackward>)
tensor(0.6685, grad_fn=<NllLossBackward>)
tensor(0.6660, grad_fn=<NllLossBackward>)
tensor(0.6788, grad_fn=<NllLossBackward>)
tensor(0.6668, grad_fn=<NllLossBackward>)
tensor(0.6674, grad_fn=<NllLossBackward>)
tensor(0.6647, grad_fn=<NllLossBackward>)
tensor(0.6795, grad_fn=<NllLossBackward>)
tensor(0.6780, grad_fn=<NllLossBackward>)
tensor(0.6839, grad_fn=<NllLossBackward>)
tensor(0.6798, grad_fn=<NllLossBackward>)
tensor(0.6689, grad_fn=<NllLossBackward>)
tensor(0.6595, grad_fn=<NllLossBac

KeyboardInterrupt: 

In [73]:
torch.from_numpy(chunks_of_y[0].values).float()

tensor([1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.])

In [77]:
test

,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,...,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,93,4,1103,1089,8,0,1,9,102,3,...,3,0,12.0,-86.0,272.0,3,4,2,0,118.0
1,20,2,556,570,1,0,0,9,6,4,...,2,2,-29.0,-54.0,1000.0,3,2,2,1,16.0
2,112,2,751,808,1,0,0,13,26,2,...,3,1,-22.0,-87.0,186.0,1,3,3,0,-34.0
3,27,3,708,903,1,1,1,11,91,2,...,2,2,-49.0,-89.0,210.0,3,4,2,1,-26.0
4,39,4,1259,661,4,0,0,9,93,5,...,2,0,36.0,-86.0,180.0,1,3,2,1,-33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17172,11,5,2054,1941,27,0,1,8,28,4,...,4,1,8.0,-87.0,1000.0,4,3,2,1,-33.0
17173,3,3,748,605,1,0,0,12,22,3,...,2,1,-1.0,-85.0,184.0,2,3,4,1,-18.0
17174,85,2,575,499,0,0,0,8,102,3,...,2,0,20.0,-88.0,239.0,4,4,4,0,-36.0
17175,7,4,1844,1176,8,1,2,8,32,3,...,2,0,-28.0,-83.0,1000.0,1,4,1,0,1000.0


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testset:
        X,y = data
        output = net(X.view(-1,784))
        for idx , i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct+=1
            total +=1 
print(correct/total)